## Creating a SoftMax Neural Net from Scratch

In [35]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plot 
import seaborn 
import tensorflow as tf

In [36]:
#Load the data
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, y_train, x_test, y_test = x_train.astype('int'), y_train.astype('int'), x_test.astype('int'), y_test.astype('int')

In [26]:
class Neural_Net_Softmax():
    
    #Initialise with the input shape and output shape
    def __init__(self, input_features, output_features):
        self.input_dimension = input_features
        self.output_dimensions = output_features

    def init_weights(self,array_of_mid_layer_sizes, array_of_mid_layer_activations, init_method='gaussian'):
        self.dict_of_weights = {}
        all_layers = [self.input_dimension] + array_of_mid_layer_sizes + [self.output_dimensions] #
        
        for layer in range(len(array_of_mid_layer_sizes) - 1):
            output_dim = all_layers[layer+1]
            input_dim = all_layers[layer] 

            #Gaussian, Xavier or He weights initialisation methods below
            if init_method == 'gaussian': 
                self.dict_of_weights[f'W{layer+1}'] = np.random.randn(output_dim,input_dim)
                self.dict_of_weights[f'B{layer+1}'] = np.random.randn(output_dim,1)
            
            if init_method == 'xavier': 
                self.dict_of_weights[f'W{layer+1}'] = np.random.uniform(-np.sqrt(6)/(np.sqrt(output_dim+input_dim)),np.sqrt(6)/(np.sqrt(output_dim+input_dim)),[output_dim,input_dim])
                self.dict_of_weights[f'B{layer+1}'] = np.random.uniform(-np.sqrt(6)/(np.sqrt(output_dim+1)),np.sqrt(6)/(np.sqrt(output_dim+1)),[output_dim,1])

            ##ADD IN THE 'HE' for relu
        return self.dict_of_weights
    
    def forward_prop(self, x_flattened, one_hot_y, dict_of_prev_values ={}):

    #Helper functions defining sigmod and relu activation functions.
    def _sigmoid(x):
        return (1/(1+np.exp(-x)))
    def _sigmoid_fp(prev_a, W, B):
        Z = W @ prev_a + B
        return Z, _sigmoid(Z)
    
    def _relu(x):
        return(max(0,x))
    def _relu_fp(prev_a, W, B):
        Z = W @ prev_a + B
        return Z, _relu(Z)
        

In [33]:
np.var(nn_1.dict_of_weights['W1'])

0.14842238536580632